In [1]:
#READ DELIVERY SCHEDULE

import os
import pandas as pd
import numpy as np
import math
import data_extract_Sheets as extr

#DELETE TRAILERMAP.XLSX TO PREVENT ADDING ON TO OLD DATA
#os.remove("TrailerMap.xlsx")

#def get_trucks_data():
   # """Gets list of trucks from current week Boa Warehousing Delivery Schedule Google Sheet.
   # """
#    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
#    creds = ServiceAccountCredentials.from_json_keyfile_name('db/reefer.json', scope)
#    client = gspread.authorize(creds)
#    sheet = client.open('Warehousing Sheet Data Feed')
#    data = sheet.worksheet('EXPORT')
#    trucks = data.get_all_values()
#    return trucks


trucklist = extr.get_trucks_data('Warehousing Sheet Data Feed', 'delivery_map')


#DEFINE COLUMN NUMBERS
wt_col = 9
plt_col = 7
shipper_col = 3
c_city_col = 5
po_col = 14


################## TRUCK NUMBER - CHANGE THIS #####################
trucknum = 18
###################################################################

truckgap = 21

#SHEET INDEX RANGE FOR EACH TRUCK
start = 7 + truckgap * (trucknum-1)
end = 23 + truckgap * (trucknum-1)
#truckindex = 1

#START WHILE LOOP

#while numberoftrucks > 0:

#Change Index Range for Different Trucks
dfi = pd.DataFrame(trucklist[start:end])
#print(df)

weight = pd.to_numeric(dfi[wt_col])
pallet = pd.to_numeric(dfi[plt_col])
wtperplt = (weight/pallet).apply(np.ceil)
#print(wtperplt)
dfi['Wt/Plt'] = wtperplt

label = dfi[shipper_col] + " " + dfi[c_city_col] + " " + dfi[po_col]
#print(label)
dfi['Label'] = label

#print(dfi)

dfi.to_excel("Trailer_Map.xlsx") 

#os.startfile("Trailer_Map.xlsx")


#MAPPING

#import os
#import pandas as pd
#from openpyxl.workbook import Workbook

#Create Data Frame to read Truck Delivery Schedule
df = pd.DataFrame(columns=['wt_L','label_L','label_R','wt_R'], index = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])
df1 = pd.read_excel("Trailer_Map.xlsx")
index = 1
index1 = 1
index2 = 1
pallet_sum = int(df1[plt_col].sum())
num_col = int(df1[po_col].count())
#print(num_col)


#Map Trailer
while num_col > 0:
    #print(num_col)
    wt = int(df1['Wt/Plt'].iloc[num_col-1])
    label = df1['Label'].iloc[num_col-1]

    pallet = int(df1[plt_col].iloc[num_col-1])
    #print(pallet)
    while pallet > 0:
        #print(index) 
        if (index%2) == 0: 

            df['wt_L'].iloc[index1-1]=wt
            df['label_L'].iloc[index1-1]=label
            pallet -= 1
            index1 += 1
            #print(wt)
            
            
        else:

            df['wt_R'].iloc[index2-1]=wt
            df['label_R'].iloc[index2-1]=label
            pallet -= 1
            index2 += 1
            #print(index2,label)
        index += 1
    num_col -= 1
            

#print(df)


#Save Trailer Map
df.to_excel('TrailerMap.xlsx')

print('Truck Mapped! File Saved')


#insert columns
from openpyxl import load_workbook
from openpyxl.styles import Border, Side, Alignment, Font
#font = Font(name='Inconsolata', size = 24,bold = True)
#thin_border = Border(left=Side(style='thin'), 
#                     right=Side(style='thin'), 
#                     top=Side(style='thin'), 
#                     bottom=Side(style='thin'))
#alignment = Alignment(horizontal='general',vertical='center',wrap_text=True) #shrink_to_fit=True,


wb = load_workbook(filename = 'TrailerMap.xlsx')
ws = wb.active
ws.insert_cols(4)
ws.insert_cols(6)

#merge cells
cell = 18
while cell > 0:
    cell_str = str(cell)
    ws.merge_cells('C' + cell_str + ':D' + cell_str)
    ws.merge_cells('E' + cell_str + ':F' + cell_str)
    cell -= 1
print('Cells Merged')

#cell_col = 7
#cell_row = 18
#while cell_row > 0:
#    while cell_col > 0:
#        ws.cell(row = cell_row, column = cell_col).border = thin_border
#        ws.cell(row = cell_row, column = cell_col).font = font
#        ws.cell(row = cell_row, column = cell_col).alignment = alignment
#        cell_col -= 1
#    cell_col = 7
#    cell_row -= 1
    
    
#print('Cells Formatted')



wb.save(filename = 'TrailerMap.xlsx')


os.startfile('TrailerMap.xlsx')



Truck Mapped! File Saved
Cells Merged


In [ ]:
from data_extract import open_sheet
import gspread
from datetime import date, datetime, timedelta
import pandas as pd


today = date.today()
year, week_num, day_of_week = today.isocalendar()
week_num_str = str(week_num) 

df3 = pd.read_excel("TrailerMap.xlsx")
df3 = df3.fillna(0)
#SRC = open_sheet('BOA Warehousing 2.0', 'Week 30 Trailer Mapping')
SRC = open_sheet('TrailerMapTest', 'Test')


rowindex = 0
truckindex = 0
mapgap = 45

time.sleep(5)

while truckindex < trucknum:

    while rowindex < 15:
        
        SRC.update_cell(rowindex+mapgap*truckindex+31,7,df3["label_R"].iloc[rowindex])
        SRC.update_cell(rowindex+mapgap*truckindex+31,9,df3["wt_R"].iloc[rowindex])
        SRC.update_cell(rowindex+mapgap*truckindex+31,4,df3["wt_L"].iloc[rowindex])
        SRC.update_cell(rowindex+mapgap*truckindex+31,5,df3["label_L"].iloc[rowindex])

        rowindex += 1
    rowindex = 0
    truckindex += 1


In [15]:
from data_extract import open_sheet
import gspread
import time
from datetime import date, datetime, timedelta
import pandas as pd


today = date.today()
year, week_num, day_of_week = today.isocalendar()
week_num_str = str(week_num) 

df3 = pd.read_excel("TrailerMap.xlsx")
df3 = df3.fillna(0)
#SRC = open_sheet('BOA Warehousing 2.0', 'Week ' + week_num_str + ' Trailer Mapping')
SRC = open_sheet('TrailerMapTest', 'Test')


####################################
truckindex = trucknum
####################################

rowindex = 0
mapgap = 45

time.sleep(5)

while truckindex < trucknum:

    while rowindex < 15:
        
        SRC.update_cell(rowindex+mapgap*truckindex+31,7,df3["label_R"].iloc[rowindex])
        SRC.update_cell(rowindex+mapgap*truckindex+31,9,df3["wt_R"].iloc[rowindex])
        SRC.update_cell(rowindex+mapgap*truckindex+31,4,df3["wt_L"].iloc[rowindex])
        SRC.update_cell(rowindex+mapgap*truckindex+31,5,df3["label_L"].iloc[rowindex])

        rowindex += 1
    rowindex = 0
    truckindex += 1



nan

NameError: name 'week_num' is not defined